In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
def glassdoor_review_scrape(email, password, review_page_1_url):
    driver.get('https://www.glassdoor.co.uk/profile/login_input.htm')
    email_address = driver.find_element_by_id('userEmail')
    password = driver.find_element_by_id('userPassword')
    email_address.send_keys(email)
    password.send_keys(password)
    submit_login = driver.find_element_by_name('submit').click()
    success  = WebDriverWait(driver, 10).until(presence_of_element_located((By.XPATH, "//*[contains(text(), 'Your Glassdoor Dashboard')]")))

    driver.get(review_page_1_url)
    review_id = 0
    all_records = []
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

    while True:
        sleep(2) #temporary measure to avoid stale elements; needs revision
        waited  = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "[class^='v2__EIReviewDetailsV2__fullWidth']")))
        waited_2  = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, './/div[@class="author minor"]')))
        reviews = driver.find_elements_by_class_name('gdReview')
        for review in reviews:
            review_date = review.find_element_by_xpath('.//time').text
            score = review.find_element_by_class_name('v2__EIReviewsRatingsStylesV2__ratingInfo').text
            pros = review.find_elements_by_css_selector("[class^='v2__EIReviewDetailsV2__fullWidth']")[0].text
            cons = review.find_elements_by_css_selector("[class^='v2__EIReviewDetailsV2__fullWidth']")[1].text
            review_id += 1
            job_status = review.find_element_by_xpath('.//div[@class="author minor"]').text
            employment_duration = review.find_element_by_xpath('.//p[@class="mainText mb-0"]').text
            record = [review_date, score, pros, cons, review_id, job_status, employment_duration]
            all_records.append(record)

        next_button = driver.find_element_by_xpath('//button[@class="nextButton css-sed91k"]')
        if next_button.get_property('disabled') == True:
            break
        if next_button.get_property('disabled type') == True:
            break


        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        next_button.click()
        waited_3  = WebDriverWait(driver, 10).until(presence_of_element_located((By.XPATH, '//button[@class="nextButton css-sed91k"]')))

print(all_records)

#driver.quit()

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['11 January 2021', '5.0\n★★★★★', 'Pros\nGood work-life balance for a consulting firm. Good salaries and rapid promotions. Amazing professionals and people. Company takes very well care of its employees.', 'Cons\nToo specialized in procurement and too much travel', 1, 'Former Employee - Consultant in London, England', 'I worked at Efficio full-time for more than 3 years'], ['6 February 2021', '3.0\n★★★★★', 'Pros\nFinally starting to move forward with modern cloud technology A good amount of freedom to make some choices like which frameworks to use A good opportunity to learn new technologies and solve challenging problems', 'Cons\nSenior leadership have little technical understanding, no CTO or head of Engineering to make decisions Due to lack of technical leadership, lots of bad short term decisions are made which increasingly build up technical debt Ceiling for technical people, you can only progress above senior level if you are like a consultant Lack of DevOps people, mostly devel

In [26]:
import pandas as pd
df = pd.DataFrame.from_records(all_records, columns=['Date', 'Rating', 'Pros', 'Cons', 'Index', 'Job title', 'Employment duration'])

df['Pros'] = df['Pros'].map(lambda x: x.lstrip('Pros\n'))
df['Cons'] = df['Cons'].map(lambda x: x.lstrip('Cons\n'))

df['Pros'].head(20)


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0     Good work-life balance for a consulting firm. ...
1     Finally starting to move forward with modern c...
2     The people and the culture is the reason to wo...
3     - Good place to build core analytical skills (...
4                  Big clients and interesting projects
5     Work-life balance, good work culture, frequent...
6     Great working environment with talented collea...
7      Location, Flexibility, Project Exposure, Clients
8     Great team members - high standards Senior exp...
9     Very dynamic place, challenging projects, supp...
10    Large pool of fun co-workers Good socials Comp...
11    *young team and relaxed company culture *compe...
12                             The people are very good
13    - People from all over the world - Growth oppo...
14                        Fast paced, high quality, fun
15         Hard to justify a lot of the work repetitive
16               Great culture\nGreat work life balance
17                           Flexible on work fr

In [27]:
import re
df['Pros'] = df['Pros'].map(lambda x: re.sub('[,\!?]', '', x))
df['Pros'] = df['Pros'].map(lambda x: x.lower())
df.head()
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:2: DeprecationWarning: invalid escape sequence \!
<>:2: DeprecationWarning: invalid escape sequence \!
<ipython-input-27-f85df6fce7b1>:2: DeprecationWarning: invalid escape sequence \!
  df['Pros'] = df['Pros'].map(lambda x: re.sub('[,\!?]', '', x))
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
pro_sentence_list = df['Pros'].tolist()
print(pro_sentence_list)
pro_sentence_list = [re.split(r'-|\.',sentence) for sentence in pro_sentence_list]
pro_sentence_list = [point for sentence in pro_sentence_list for point in sentence if point!='']
print(pro_sentence_list)
stpword = stopwords.words('english')

def sentences_to_words(sentences):
    for i in sentences:
        yield(gensim.utils.simple_preprocess(i, deacc=True))

pro_word_list = list(sentences_to_words(pro_sentence_list))


def rm_stopwords(words_list):
    return [[word for word in words if word not in stpword] for words in words_list]

pro_word_list = rm_stopwords(pro_word_list)


['good work-life balance for a consulting firm. good salaries and rapid promotions. amazing professionals and people. company takes very well care of its employees.', 'finally starting to move forward with modern cloud technology a good amount of freedom to make some choices like which frameworks to use a good opportunity to learn new technologies and solve challenging problems', 'the people and the culture is the reason to work for efficio. despite rapid growth in recent years efficio has still managed to maintain a collaborative and tight-knit environment.', "- good place to build core analytical skills (excel ppt tableau sql) - solid comp per hour worked (decent work-life balance) but you'll be paid a lot more at other companies when manager+ level - young dynamic workforce very much a party culture - good travel options if you speak a second language - if you're set on doing procurement then one of the top 3 places to be", 'big clients and interesting projects', 'work-life balance 

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

pro_word_list_lemm = lemmatization(pro_word_list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
print(pro_word_list_lemm[0])

id2word = corpora.Dictionary(pro_word_list_lemm)
texts = pro_word_list_lemm
corpus = [id2word.doc2bow(text) for text in texts]

['good', 'work']


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
from pprint import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.109*"great" + 0.092*"culture" + 0.067*"work" + 0.056*"people" + '
  '0.050*"company" + 0.034*"high" + 0.028*"employee" + 0.025*"growth" + '
  '0.025*"responsibility" + 0.023*"event"'),
 (1,
  '0.148*"work" + 0.040*"good" + 0.036*"smart" + 0.035*"colleague" + '
  '0.023*"place" + 0.021*"people" + 0.021*"environment" + 0.019*"young" + '
  '0.019*"experience" + 0.019*"great"'),
 (2,
  '0.042*"opportunity" + 0.034*"learning" + 0.033*"good" + '
  '0.033*"international" + 0.032*"nice" + 0.031*"fast" + 0.027*"people" + '
  '0.026*"lot" + 0.025*"company" + 0.025*"project"'),
 (3,
  '0.081*"management" + 0.059*"senior" + 0.045*"good" + 0.045*"people" + '
  '0.034*"super" + 0.026*"build" + 0.023*"skill" + 0.022*"bright" + '
  '0.018*"structure" + 0.015*"extremely"'),
 (4,
  '0.083*"life" + 0.078*"balance" + 0.045*"excellent" + 0.039*"make" + '
  '0.027*"manager" + 0.025*"compare" + 0.025*"good" + 0.021*"lot" + '
  '0.020*"environment" + 0.017*"impact"'),
 (5,
  '0.103*"good" + 0.056*"a

In [34]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
coherence_model_lda = CoherenceModel(model=lda_model, texts=pro_word_list_lemm, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -6.473520432328505

Coherence Score:  0.4575948972834335


In [35]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.259237  0.086399       1        1  17.818847
2      0.103991 -0.009643       2        1  15.498491
1      0.115832 -0.058476       3        1  13.204403
7     -0.047147  0.074288       4        1  11.576914
5     -0.047674 -0.121620       5        1   8.977069
9     -0.017622 -0.003742       6        1   8.141112
8     -0.145181  0.189385       7        1   7.462793
3     -0.066606 -0.146769       8        1   7.385828
4     -0.076916 -0.029066       9        1   6.481717
6     -0.077913  0.019245      10        1   3.452825, topic_info=            Term       Freq      Total Category  logprob  loglift
1           work  63.000000  63.000000  Default  30.0000  30.0000
63           lot  28.000000  28.000000  Default  29.0000  29.0000
85         great  41.000000  41.000000  Default  28.0000  28.0000
36       culture  32.000000  32.000000  Default  27.0000  27.0000
0           good  45.000000  45.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
570  transparent   0.847266   2.576267  Topic10  -4.2591   2.2539
93          high   1.246454  12.530845  Topic10  -3.8731   1.0581
415       junior   0.540615   3.305280  Topic10  -4.7084   1.5554
95        senior   0.625799   8.834997  Topic10  -4.5621   0.7185
50         build   0.401276   4.307165  Topic10  -5.0065   0.9926

[429 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
552       7  0.561148       ability
571       8  0.593983        access
550       2  0.506041  accomodation
242       6  0.523608      activity
502       6  0.539899       advance
...     ...       ...           ...
122       2  0.655532         world
46        4  0.902958          year
70        2  0.127789         young
70        3  0.511156         young
70        5  0.255578         young

[431 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 8, 6, 10, 9, 4, 5, 7])

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
